In [0]:
#install gcloud
%sh
curl -O https://dl.google.com/dl/cloudsdk/channels/rapid/downloads/google-cloud-sdk-356.0.0-linux-x86_64.tar.gz
tar -xf google-cloud-sdk-356.0.0-linux-x86_64.tar.gz
yes | ./google-cloud-sdk/install.sh

In [0]:
#add gcloud directory to PATH env var
import os
print(os.environ['PATH'])
path='/databricks/driver/google-cloud-sdk/bin'
os.environ["PATH"] += os.pathsep + os.path.join(path)
print(os.environ['PATH'])

In [0]:
#check if gcloud works with new PATH env variable
%sh
/databricks/driver/google-cloud-sdk/bin/gcloud version
gcloud version

In [0]:
import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)

import mlflow
from mlflow.deployments import get_deploy_client
print(mlflow.__version__)
client=mlflow.deployments.get_deploy_client("google_cloud")

import mlflow
import mlflow.sklearn
import pandas as pd
import matplotlib.pyplot as plt
 
from numpy import savetxt
 
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [0]:
db = load_diabetes()
X = db.data
y = db.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [0]:
# Enable autolog()
# mlflow.sklearn.autolog() requires mlflow 1.11.0 or above.
mlflow.sklearn.autolog()
 
# With autolog() enabled, all model parameters, a model score, and the fitted model are automatically logged.  
with mlflow.start_run():
  
  # Set the model parameters. 
  n_estimators = 100
  max_depth = 6
  max_features = 3
  
  # Create and train model.
  rf = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = max_features)
  rf.fit(X_train, y_train)
  
  # Use the model to make predictions on the test dataset.
  predictions = rf.predict(X_test)

In [0]:
#mchrestkha-sklearnVersion 1
model_name = "mchrestkha-test-3"
model_version = 1
model_uri=f"models:/{model_name}/{model_version}"
print(model_uri)

In [0]:
deployment = client.create_deployment(
    name="mlflow_on_gcp_test_3",
    model_uri=model_uri)

INFO:google_cloud_mlflow._mlflow_model_gcp_deployment_utils:Project not set. Using project-aa-258321 as project
INFO:google_cloud_mlflow._mlflow_model_gcp_deployment_utils:Destination image URI not set. Building and uploading image to gcr.io/project-aa-258321/mlflow/mlflow_on_gcp
INFO:google_cloud_mlflow._mlflow_model_gcp_deployment_utils:Building image. This can take up to 20 minutes
2021/09/10 06:13:41 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
INFO:google_cloud_mlflow._mlflow_models_docker_utils_patch:Building docker image with name gcr.io/project-aa-258321/mlflow/mlflow_on_gcp